# Shipment Tracker API

In [1]:
import pandas as pd

# Set display option to show all columns and max column width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### File Paths and Backup

In [2]:
from functions0_basics import (get_files_path,
                               save_backup,
                               shipments_not_delivered,
                               get_API_details, save_to_excel, convert_urls_to_links)

# Get files paths and store them into variables (chromedriver if needed)
excel_path, report_path, chromedriver_path = get_files_path("Albert")

# Create a backup of original Excel file 
save_backup(excel_path)

Backup saved to: ./Shipment Data/BackUp - Daily Report Shipments of Spare Parts.xlsx 26-12-2023 20_59_10.xlsx
Deleted old backup: ./Shipment Data/BackUp - Daily Report Shipments of Spare Parts.xlsx 26-12-2023 20_37_18.xlsx


In [3]:
# Get data for all carriers != 'Delivered'
shipments_not_delivered = shipments_not_delivered(excel_path)

**150 shipments NOT DELIVERED in your file**

Status,Carrier,In Transit,Exception,Totals
0,DHL,16,0,16
1,NACEX,2,1,3
2,SEUR,1,1,2
3,TNT,127,2,129


### TNT

In [4]:
# TNT: API Requests
from functions1_TNT_requests import make_tnt_requests
tnt_results, len_shipm_numbers = make_tnt_requests(shipments_not_delivered)

In [5]:
# TNT: Extract, process and create dataframe
from functions2_TNT_dataframe import tnt_to_dataframe
tnt_df = tnt_to_dataframe(tnt_results, shipments_not_delivered, len_shipm_numbers, report_path)
# Saved df:
#tnt_df

# Linkable df:
# tnt_df_linkable = convert_urls_to_links(tnt_df)

Successfully retrieved all TNT shipments data in attempt 1.
 


New report file saved at: Track Reports/TNT - Track Report 26-12-2023 20_59_17.xlsx

### DHL

In [6]:
# DHL: API Requests
from functions1_DHL_requests import make_dhl_requests
all_dhl_results, max_dhl_shipm = make_dhl_requests(shipments_not_delivered)

Rate limited for URL https://api-eu.dhl.com/track/shipments?trackingNumber=3746393210,5581782161,1465781925. Retrying each shipment individually.
Error 429 for https://api-eu.dhl.com/track/shipments?trackingNumber=5581782161: Too many requests within defined time period, please try again later.


In [7]:
# DHL: Extract, process and create dataframe
from functions2_DHL_dataframe import dhl_to_dataframe
dhl_df = dhl_to_dataframe(all_dhl_results, shipments_not_delivered, max_dhl_shipm, report_path)
# Saved df:
#dhl_df

# Linkable df:
# dhl_df_linkable = convert_urls_to_links(dhl_df)

Missing DHL data in attempt 1.
Missing DHL data in attempt 2.
Missing DHL data in attempt 3.
Attempt: 3 / 3
Maximum attempts reached. Could not retrieve all DHL shipments data.

Missing DHL data shipments URL: 
https://www.dhl.com/es-en/home/tracking/tracking-express.html?submit=1&tracking-id=5581782161
 


New report file saved at: Track Reports/DHL - Track Report 26-12-2023 20_59_27.xlsx